In [70]:
import yfinance as yf
import pandas as pd
import certifi
import ssl
import urllib.request
from concurrent.futures import ThreadPoolExecutor, as_completed
from sqlalchemy import create_engine
import psycopg2

In [54]:
# Define the ticker symbol
ticker_symbol = 'AAPL'

# Fetch the ticker data
ticker = yf.Ticker(ticker_symbol)

# Get historical market data (trading info)
trading_info = ticker.history(period="1d",interval="1m")  # You can change the period ("1d", "5d", "1mo", "3mo", etc.)
trading_info = trading_info.reset_index()
print(len(trading_info))
# Display the trading information
trading_info.head()

330


,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-10-25 09:30:00-04:00,229.740005,230.550003,229.570007,230.350006,1583225,0.0,0.0
1,2024-10-25 09:31:00-04:00,230.360001,231.119995,230.195007,230.865005,292564,0.0,0.0
2,2024-10-25 09:32:00-04:00,230.860001,231.289993,230.780106,230.910004,188562,0.0,0.0
3,2024-10-25 09:33:00-04:00,230.938995,231.339996,230.889999,231.333801,150528,0.0,0.0
4,2024-10-25 09:34:00-04:00,231.279999,231.369705,231.020096,231.110001,124380,0.0,0.0


In [93]:
# Set up SSL context using certifi's certificates
ssl_context = ssl.create_default_context(cafile=certifi.where())

# Scrape the S&P 500 list from Wikipedia using urllib with the SSL context
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = urllib.request.urlopen(url, context=ssl_context)
html = response.read()

# Use pandas to parse the HTML and extract the table
table = pd.read_html(html)

# Extract the first table (which contains the ticker symbols)
sp500_df = table[0]

# Get the list of ticker symbols
tickers = sp500_df['Symbol'].tolist()

# Show the first 5 tickers as a preview
tickers[:5]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']

In [94]:
def fetch_ticker_data(ticker_symbol,period="1d",interval="1m"):

    # Fetch the ticker data
    ticker = yf.Ticker(ticker_symbol)

    # Get historical market data (trading info)
    trading_info = ticker.history(period=period,interval=interval)  # You can change the period ("1d", "5d", "1mo", "3mo", etc.)
    trading_info['ticker'] = ticker_symbol

    return trading_info

In [89]:
len(fetch_ticker_data('AAPL'))

379

In [34]:
all_data = pd.DataFrame()
for ticker in tickers:
    try:
        trading_info = fetch_ticker_data(ticker)
        all_data = pd.concat([all_data, trading_info],ignore_index=True)
    except:
        print(f"Failed to fetch data for {ticker}")

print(all_data)

              Open        High         Low       Close    Volume  Dividends  \
0       219.860001  220.419998  218.617004  220.125000         0        0.0   
1       220.100006  220.270004  217.330002  217.490005   5667186        0.0   
2       218.850006  219.149902  216.009995  217.399994  10822541        0.0   
3       217.395004  218.154007  216.455902  217.955399   4349192        0.0   
4       217.955002  219.020004  217.509995  218.691193   4508080        0.0   
...            ...         ...         ...         ...       ...        ...   
229363  232.119995  233.199997  232.119995  232.229996   4138746        0.0   
229364  232.229996  232.731995  231.759995  232.710007   3211126        0.0   
229365  232.735001  232.940002  231.529999  231.857498   2367087        0.0   
229366  231.830002  231.971497  231.070007  231.839996   2818009        0.0   
229367  231.846893  232.119995  231.669998  232.035004    699856        0.0   

        Stock Splits Ticker  
0                0.0 

In [95]:
def fetch_data_parallel(tickers, max_workers=10):
    all_data = pd.DataFrame()

    # Use ThreadPoolExecutor to fetch data in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit the tasks to the thread pool
        futures = {executor.submit(fetch_ticker_data, ticker): ticker for ticker in tickers}

        # As the tasks complete, process the results
        for future in as_completed(futures):
            ticker = futures[future]
            try:
                # Get the result of the completed task
                trading_info = future.result()
                trading_info.reset_index(inplace=True)
                if not trading_info.empty:  # Only append if data is returned
                    all_data = pd.concat([all_data, trading_info],ignore_index=True)
            except Exception as e:
                print(f"Error processing data for {ticker}: {e}")

    return all_data

In [96]:
all_data = fetch_data_parallel(tickers, max_workers=10)
all_data.columns = all_data.columns.str.lower()
print(len(all_data))
all_data.drop_duplicates(subset=['datetime', 'ticker'], keep='first',inplace=True)
print(len(all_data))

$BF.B: possibly delisted; no price data found  (period=1d)
$BRK.B: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


168242
168242


In [97]:
memory_usage = all_data.memory_usage(deep=True).sum()/(1024**2)
print(memory_usage)
print(all_data)
print(type(all_data['datetime'][0]))

19.920597076416016
                        datetime        open        high         low  \
0      2024-10-25 09:30:00-04:00  189.679993  189.850006  189.000000   
1      2024-10-25 09:31:00-04:00  189.679993  189.699997  189.419998   
2      2024-10-25 09:32:00-04:00  189.669998  189.880005  189.589996   
3      2024-10-25 09:33:00-04:00  189.660004  189.660004  189.130005   
4      2024-10-25 09:34:00-04:00  189.404999  189.500000  189.220001   
...                          ...         ...         ...         ...   
168237 2024-10-25 15:55:00-04:00  360.670013  360.730011  360.380005   
168238 2024-10-25 15:56:00-04:00  360.450012  360.450012  360.304993   
168239 2024-10-25 15:57:00-04:00  360.420013  360.420013  360.109985   
168240 2024-10-25 15:58:00-04:00  359.940002  359.950012  359.880005   
168241 2024-10-25 15:59:00-04:00  359.934998  360.079987  359.829987   

             close  volume  dividends  stock splits ticker  
0       189.639999   96574        0.0           0.0   A

In [98]:
db_user = 'ambarchakraborty'
db_password = 'ChakraA%402706'
db_host = 'localhost'
db_port = '5432'
db_name = 'yfinance'

# Create SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [99]:
all_data.to_sql('stocks', engine, if_exists='append', index=False)

242